Debajo, se importan los tweets y se les agrega un campo nuevo 'longitud'

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#import geopy as gp
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import datetime

%pylab inline

tweets = pd.read_csv('train.csv', encoding='latin-1')
tweets['longitud'] = tweets['text'].str.len()
tweets.head(10)

Ahora tomamos los tweets reales, como muestra para demostrar el desarrollo del algoritmo y los problemas encontrados
Luego se hará el mismo procesamiento pero para todos los tweets cuya "location" no sea nula

In [ ]:
true_tweets = tweets[tweets.target==1]

In [ ]:
true_tweets.count()

Ahora, de esos tweets reales, se toman los que tengan datos de localización

In [ ]:
true_tweets_with_loc = true_tweets[true_tweets.location.isnull()==False]

In [ ]:
true_tweets_with_loc.info()

In [ ]:
true_tweets_with_loc_sub = true_tweets_with_loc.head(10)
geolocator = Nominatim(user_agent = "TP1 1C2020 7506 FIUBA - Barbijo_de_uranio")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds = 0.1)
true_tweets_with_loc_sub = true_tweets_with_loc_sub.assign(geolocation = true_tweets_with_loc_sub['location'].\
                                                           apply(geocode))

In [ ]:
true_tweets_with_loc_sub

In [ ]:
true_tweets_with_loc_sub = true_tweets_with_loc_sub[true_tweets_with_loc_sub.geolocation.isnull()==False]
true_tweets_with_loc_sub

In [ ]:
true_tweets_with_loc_sub.info()

Ahora se arma una lista de países, tomando el campo "address" de cada geolocalización

In [ ]:
country_list = []
for row in true_tweets_with_loc_sub.iterrows():
    try:
        country_list.append(geolocator.geocode(row[1].location).address.split(", ")[-1])
    except:
        country_list.append(None)

La lista de países así obtenida produce resultados distintos para un mismo país (ver caso de USA debajo)

In [ ]:
country_list

In [ ]:
true_tweets_with_loc_sub['country'] = country_list

In [ ]:
true_tweets_with_loc_sub

In [ ]:
true_tweets_with_loc_sub.reset_index(drop=True, inplace=True)
true_tweets_with_loc_sub

Debajo se ve que en vez de resultar 4 tweets de USA, hay 3 por un lado, y 1 por otro, ya que el "país" obtenido
es distinto por cómo está cargado en el campo "address" de la geolocalizaicón correspondiente

In [ ]:
true_tweets_with_loc_sub.country.value_counts()

Para obtener un nombre de país único, se nota que convirtiendo la "location" a coordenadas primero, y luego
haciendo un reverse lookup de esas coordenadas de la cual se extrae la "address" resultante, el país ahora
sí tiene nombre único

In [ ]:
lat_list = []
long_list = []
for row in true_tweets_with_loc_sub.iterrows():
    try:
        point = (geolocator.geocode(row[1].location))
        lat_list.append(point.latitude)
        long_list.append(point.longitude)
    except:
        lat_list.append(None)
        long_list.append(None)

In [ ]:
lat_list

In [ ]:
long_list

In [ ]:
true_tweets_with_loc_sub['latitude'] = lat_list

In [ ]:
true_tweets_with_loc_sub['longitude'] = long_list

In [ ]:
true_tweets_with_loc_sub

In [ ]:
true_tweets_with_loc_sub['coordinates'] = true_tweets_with_loc_sub['latitude'].astype('str') + ', ' +\
                                          true_tweets_with_loc_sub['longitude'].astype('str')

In [ ]:
true_tweets_with_loc_sub

In [ ]:
true_tweets_with_loc_sub['coordinates'][0]

En el ejemplo debajo, se ve que ahora "country" en vez de ser "United States" como antes, ahora sí es "United Sates of
America", lo cual hará que el conteo resulte 4 como debía ser

In [ ]:
geolocator.reverse(true_tweets_with_loc_sub['coordinates'][7]).address.split(", ")[-1]

In [ ]:
country_from_coord_list = []
for row in true_tweets_with_loc_sub.iterrows():
    try:
        country_from_coord_list.append(geolocator.reverse(\
                        row[1].coordinates).address.split(", ")[-1])
    except:
        country_from_coord_list.append(None)

In [ ]:
country_from_coord_list

In [ ]:
true_tweets_with_loc_sub['country from coordinates'] = country_from_coord_list

In [ ]:
true_tweets_with_loc_sub

Ahora sí, la cuenta de países de la muestra da bien

In [ ]:
true_tweets_with_loc_sub['country from coordinates'].value_counts()

# Ahora se hará el mismo procesamiento pero para todos los tweets cuya "location" no sea nula, independientemente de si los tweets son reales o falsos

In [ ]:
tweets_with_loc = tweets[tweets.location.isnull()==False]

In [ ]:
tweets_with_loc.reset_index(drop=True, inplace=True)
tweets_with_loc.info()

In [ ]:
tweets_with_loc.head(20)

In [ ]:
print(datetime.datetime.now())

Para obtener un nombre de país único, se nota que convirtiendo la "location" a coordenadas primero, y luego haciendo un reverse lookup de esas coordenadas de la cual se extrae la "address" resultante, el país ahora sí tiene nombre único

In [ ]:
lat_list = []
long_list = []
for row in tweets_with_loc.iterrows():
    try:
        point = (geolocator.geocode(row[1].location))
        lat_list.append(point.latitude)
        long_list.append(point.longitude)
    except:
        lat_list.append(None)
        long_list.append(None)

In [ ]:
print(datetime.datetime.now())

In [ ]:
tweets_with_loc['latitude'] = lat_list

In [ ]:
tweets_with_loc['longitude'] = long_list

In [ ]:
tweets_with_loc.head(20)

In [ ]:
tweets_with_loc['coordinates'] = tweets_with_loc['latitude'].astype('str') + ', ' +\
                                          tweets_with_loc['longitude'].astype('str')

In [ ]:
tweets_with_loc

In [ ]:
print(datetime.datetime.now())

In [ ]:
country_from_coord_list = []
for row in tweets_with_loc.iterrows():
    try:
        country_from_coord_list.append(geolocator.reverse(\
                        row[1].coordinates).address.split(", ")[-1])
    except:
        country_from_coord_list.append(None)

In [ ]:
print(datetime.datetime.now())

In [ ]:
tweets_with_loc['country from coordinates'] = country_from_coord_list

In [ ]:
tweets_with_loc

Ahora se cuentan los países

In [ ]:
tweets_with_loc['country from coordinates'].value_counts()

In [ ]:
tweets_with_loc.to_csv(r'F:\Documentos\Pablo\facultad\orga_de_datos\TP1-mio\tweets_with_loc.csv', index = False)